In [ ]:
# Load Dependencies

import pandas as pd
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [ ]:
# Read in data and store them as a dictionary

file_names = [
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_.5s_epoch_corr_df_no_nans.csv',
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_1s_epoch_corr_df_no_nans.csv',
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_1.5s_epoch_corr_df_no_nans.csv',
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_2s_epoch_corr_df_no_nans.csv',
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_2.5s_epoch_corr_df_no_nans.csv',
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_3s_epoch_corr_df_no_nans.csv',
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_3.5s_epoch_corr_df_no_nans.csv',
    'preprocessed_interpolate_bandpass_.1_20/merged_corr_df_4s_epoch_corr_df_no_nans.csv'
]

dataframes = {name: pd.read_csv(name) for name in file_names}


In [ ]:
#Rename "Time (sec)" to "Time"

for name, df in dataframes.items():
    # Rename the column 'Time (sec)' to 'Time' in each DataFrame
    df = df.rename(columns={'Time (sec)': 'Time'})
    dataframes[name] = df  # Update the dictionary with the renamed DataFrame

In [ ]:
# Function to extract the epoch length from the filename
def extract_epoch_length(filename):
    # The regex looks for a potential leading dot (for .5s) and then digits
    match = re.search(r'_((?:\.\d+)|\d+(\.\d+)?)s_epoch', filename)
    if match:
        # If the match starts with a dot, prepend a 0, otherwise return the match as a float
        return float(match.group(1)) if not match.group(1).startswith('.') else float('0' + match.group(1))
    else:
        raise ValueError(f"Epoch length not found in file name: {filename}")

In [ ]:
# Combine all files into one DataFrame with an additional 'Epoch' column

all_data = []
for file in file_names:
    df = pd.read_csv(file)
    epoch_length = extract_epoch_length(file)
    df['Epoch'] = epoch_length
    all_data.append(df)

In [ ]:
# Concatenate all dataframes into one
combined_df = pd.concat(all_data)

In [ ]:
# Define custom colors for each electrode
# comes from https://colorbrewer2.org/

electrode_colors = {
    'AF7': '#bfd3e6',
    'AF8': '#8c96c6',
    'TP9': '#88419d',
    'TP10': '#4d004b'
}

# Plot configuration
small_font_size = 6
title_font_size = 6
fig_suptitle_font_size = 6
line_width = 0.5
box_width = 0.5
outlier_marker_size = 1.0

num_sessions = combined_df['ses'].nunique()
num_epochs = len(file_names)
fig, axes = plt.subplots(nrows=num_epochs, ncols=num_sessions, figsize=(8.5, 11), dpi=300, sharey=True)

# Plot boxplots with customized colors for each electrode
for i, epoch in enumerate(sorted(combined_df['Epoch'].unique())):
    for j, session in enumerate(sorted(combined_df['ses'].unique())):
        ax = axes[i][j] if num_epochs > 1 else axes[j]
        # Extract data for the current epoch and session
        epoch_data = combined_df[(combined_df['Epoch'] == epoch) & (combined_df['ses'] == session)]
        # Melt the data to long-form for plotting with seaborn
        melted_data = epoch_data.melt(value_vars=['AF7', 'AF8', 'TP9', 'TP10'], var_name='Electrode', value_name='ISC')
        sns.boxplot(x='Electrode', y='ISC', data=melted_data, ax=ax, linewidth=line_width, width=box_width,
                    fliersize=outlier_marker_size, palette=electrode_colors)
        ax.set_ylim(-0.8, 0.8)
        ax.set_title(f'ISCs for Epoch Size {epoch}s - Session {session}', fontsize=title_font_size)
        ax.set_xlabel('')
        ax.set_ylabel('')
        ax.tick_params(labelsize=small_font_size)

plt.tight_layout(rect=[0, 0, 1, 0.95], pad=0.4, h_pad=0.5, w_pad=0.5)
plt.savefig('plots/multiverse_isc_boxplots.png', format='png', bbox_inches='tight', facecolor='white')
plt.show()


In [ ]:
# Storage for multiverse model summaries

param_estimates = pd.DataFrame()
p_values = pd.DataFrame()

In [ ]:
# For PERFORMANCE

# Apply different filtering thresholds for correlation values (ranging from 0.1 to 1.0 in .1 increments)
# Then, for each filtered threshold, fit mixed models
# Then store parameter estimates and p-values in the new dataframes defined above

# Thresholds
thresholds = [x * 0.1 for x in range(1, 11)]  # 0.1 to 1.0 in increments of 0.1

for name, df in dataframes.items():
    for threshold in thresholds:
        filter_index = ((df[["TP10", "AF7", "AF8", "TP9"]] > threshold).any(1) | 
                        (df[["TP10", "AF7", "AF8", "TP9"]] < -threshold).any(1) | 
                        (df["Signal Drop"] == 1))
        filtered_df = df[~filter_index]

        # Loop through each electrode to fit a model
        for electrode in ['AF7', 'AF8', 'TP9', 'TP10']:
            model = smf.mixedlm(f"Performance ~ {electrode}", data=filtered_df, groups=filtered_df["subj"],
                                re_formula="1", vc_formula={"ses": "0 + C(ses)"})
            results = model.fit()

            # Store parameter estimates and p-values
            param_key = f"{name}_{threshold}_{electrode}"
            param_estimates.loc[param_key, 'Intercept'] = results.params['Intercept']
            param_estimates.loc[param_key, 'Electrode'] = electrode
            param_estimates.loc[param_key, 'Estimate'] = results.params[electrode]
            
            p_values.loc[param_key, 'Intercept_p'] = results.pvalues['Intercept']
            p_values.loc[param_key, 'Electrode'] = electrode
            p_values.loc[param_key, 'p_value'] = results.pvalues[electrode]

            # Print model summaries
            print(f"Results for {name}, Threshold {threshold}, Electrode {electrode}")
            print(results.summary())

In [ ]:
# Multiverse plot for all electrodes for performance

# Function to extract the numeric part from dataset name
def extract_numeric_part(dataset_name):
    match = re.search(r'(\d+(\.\d+)?|\.5)s', dataset_name)
    if match:
        return float(match.group(1))
    return 0

# Extract dataset names and thresholds
try:
    dataset_names = set('_'.join(idx.split('_')[:-2]) for idx in param_estimates.index)
    dataset_names = sorted(dataset_names, key=extract_numeric_part)
    thresholds = sorted(set(float(idx.split('_')[-2]) for idx in param_estimates.index))
except ValueError as e:
    print(f"Error parsing index: {e}")
    print("Sample problematic indices:", param_estimates.index[:5])

# Define font sizes for the plot
tiny_font_size = 6
smaller_font_size = 8  # For super title and column titles

fig, axes = plt.subplots(nrows=len(dataset_names), ncols=2, figsize=(8.5, 11), dpi=300)
fig.subplots_adjust(hspace=0.4, wspace=0.3)
#fig.suptitle("Multiverse: ISC by Electrode", fontsize=smaller_font_size, va='bottom')

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']
colors = ['#bfd3e6', '#8c96c6', '#88419d', '#4d004b']  # colors from https://colorbrewer2.org/

for i, dataset in enumerate(dataset_names):
    epoch_value = f"{extract_numeric_part(dataset)}s"
    for j, df in enumerate([param_estimates, p_values]):
        ax = axes[i, j]
        for electrode, color in zip(electrodes, colors):
            filtered_data = df[df['Electrode'] == electrode]
            filtered_data = filtered_data[filtered_data.index.str.contains(dataset)]
            filtered_data['Threshold'] = filtered_data.index.map(lambda x: float(x.split('_')[-2]))

            y_column = 'Estimate' if 'Estimate' in df.columns else 'p_value'
            sns.lineplot(x='Threshold', y=y_column, data=filtered_data, ax=ax, marker='o', color=color, label=electrode,
                        linewidth=1.5, markersize=4)

        ax.set_title(f"Epoch {epoch_value}", fontsize=tiny_font_size, pad=3)
        ax.set_xlabel('Filter Threshold', fontsize=tiny_font_size)
        ax.set_ylabel(y_column, fontsize=tiny_font_size)
        ax.tick_params(labelsize=tiny_font_size)
        if j == 1:  # P-value column specific adjustments
            ax.set_ylim(0, 1)
            ax.axhline(0.05, color='red', linestyle='--')
        ax.legend(title='Electrode', title_fontsize='small', fontsize='x-small', loc='upper right')

axes[0, 0].set_title("Parameter Estimate", fontsize=smaller_font_size, pad=20)
axes[0, 1].set_title("P-Value", fontsize=smaller_font_size, pad=20)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('plots/multiverse_isc_by_electrode_performance.png', format='png', bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()


In [ ]:
# Storage for multiverse model summaries

param_estimates_time = pd.DataFrame()
p_values_time = pd.DataFrame()

In [ ]:
# For TIME

# Apply different filtering thresholds for correlation values (ranging from 0.1 to 1.0 in .1 increments)
# Then, for each filtered threshold, fit mixed models
# Then store parameter estimates and p-values in the new dataframes defined above

# Thresholds
thresholds = [x * 0.1 for x in range(1, 11)]  # 0.1 to 1.0 in increments of 0.1

for name, df in dataframes.items():
    for threshold in thresholds:
        filter_index = ((df[["TP10", "AF7", "AF8", "TP9"]] > threshold).any(1) | 
                        (df[["TP10", "AF7", "AF8", "TP9"]] < -threshold).any(1) | 
                        (df["Signal Drop"] == 1))
        filtered_df = df[~filter_index]

        # Loop through each electrode to fit a model
        for electrode in ['AF7', 'AF8', 'TP9', 'TP10']:
            model = smf.mixedlm(f"Time ~ {electrode}", data=filtered_df, groups=filtered_df["subj"],
                                re_formula="1", vc_formula={"ses": "0 + C(ses)"})
            results = model.fit()

            # Store parameter estimates and p-values
            param_key = f"{name}_{threshold}_{electrode}"
            param_estimates_time.loc[param_key, 'Intercept'] = results.params['Intercept']
            param_estimates_time.loc[param_key, 'Electrode'] = electrode
            param_estimates_time.loc[param_key, 'Estimate'] = results.params[electrode]
            
            p_values_time.loc[param_key, 'Intercept_p'] = results.pvalues['Intercept']
            p_values_time.loc[param_key, 'Electrode'] = electrode
            p_values_time.loc[param_key, 'p_value'] = results.pvalues[electrode]

            # Print model summaries
            print(f"Results for {name}, Threshold {threshold}, Electrode {electrode}")
            print(results.summary())

In [ ]:
# Multiverse plot for all electrodes for time

# Function to extract the numeric part from dataset name
def extract_numeric_part(dataset_name):
    match = re.search(r'(\d+(\.\d+)?|\.5)s', dataset_name)
    if match:
        return float(match.group(1))
    return 0

# Extract dataset names and thresholds
try:
    dataset_names = set('_'.join(idx.split('_')[:-2]) for idx in param_estimates_time.index)
    dataset_names = sorted(dataset_names, key=extract_numeric_part)
    thresholds = sorted(set(float(idx.split('_')[-2]) for idx in param_estimates_time.index))
except ValueError as e:
    print(f"Error parsing index: {e}")
    print("Sample problematic indices:", param_estimates_time.index[:5])

# Define font sizes for the plot
tiny_font_size = 6
smaller_font_size = 8  # For super title and column titles

fig, axes = plt.subplots(nrows=len(dataset_names), ncols=2, figsize=(8.5, 11), dpi=300)
fig.subplots_adjust(hspace=0.4, wspace=0.3)
#fig.suptitle("Multiverse: ISC by Electrode", fontsize=smaller_font_size, va='bottom')

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']
colors = ['#bfd3e6', '#8c96c6', '#88419d', '#4d004b']  # colors from https://colorbrewer2.org/

for i, dataset in enumerate(dataset_names):
    epoch_value = f"{extract_numeric_part(dataset)}s"
    for j, df in enumerate([param_estimates_time, p_values_time]):
        ax = axes[i, j]
        for electrode, color in zip(electrodes, colors):
            filtered_data = df[df['Electrode'] == electrode]
            filtered_data = filtered_data[filtered_data.index.str.contains(dataset)]
            filtered_data['Threshold'] = filtered_data.index.map(lambda x: float(x.split('_')[-2]))

            y_column = 'Estimate' if 'Estimate' in df.columns else 'p_value'
            sns.lineplot(x='Threshold', y=y_column, data=filtered_data, ax=ax, marker='o', color=color, label=electrode,
                        linewidth=1.5, markersize=4)

        ax.set_title(f"Epoch {epoch_value}", fontsize=tiny_font_size, pad=3)
        ax.set_xlabel('Filter Threshold', fontsize=tiny_font_size)
        ax.set_ylabel(y_column, fontsize=tiny_font_size)
        ax.tick_params(labelsize=tiny_font_size)
        if j == 1:  # P-value column specific adjustments
            ax.set_ylim(0, 1)
            ax.axhline(0.05, color='red', linestyle='--')
        ax.legend(title='Electrode', title_fontsize='small', fontsize='x-small', loc='upper right')

axes[0, 0].set_title("Parameter Estimate", fontsize=smaller_font_size, pad=20)
axes[0, 1].set_title("P-Value", fontsize=smaller_font_size, pad=20)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('plots/multiverse_isc_by_electrode_time.png', format='png', bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()


In [ ]:
filtered_df

In [ ]:
# Storage for multiverse model summaries

param_estimates_cognitive_trust = pd.DataFrame()
p_values_cognitive_trust = pd.DataFrame()

In [ ]:
# For Cognitive Trust

# Apply different filtering thresholds for correlation values (ranging from 0.1 to 1.0 in .1 increments)
# Then, for each filtered threshold, fit mixed models
# Then store parameter estimates and p-values in the new dataframes defined above

# Thresholds
thresholds = [x * 0.1 for x in range(1, 11)]  # 0.1 to 1.0 in increments of 0.1

for name, df in dataframes.items():
    for threshold in thresholds:
        filter_index = ((df[["TP10", "AF7", "AF8", "TP9"]] > threshold).any(1) | 
                        (df[["TP10", "AF7", "AF8", "TP9"]] < -threshold).any(1) | 
                        (df["Signal Drop"] == 1))
        filtered_df = df[~filter_index]

        # Loop through each electrode to fit a model
        for electrode in ['AF7', 'AF8', 'TP9', 'TP10']:
            model = smf.mixedlm(f"cognitive_trust ~ {electrode}", data=filtered_df, groups=filtered_df["subj"],
                                re_formula="1", vc_formula={"ses": "0 + C(ses)"})
            results = model.fit()

            # Store parameter estimates and p-values
            param_key = f"{name}_{threshold}_{electrode}"
            param_estimates_cognitive_trust.loc[param_key, 'Intercept'] = results.params['Intercept']
            param_estimates_cognitive_trust.loc[param_key, 'Electrode'] = electrode
            param_estimates_cognitive_trust.loc[param_key, 'Estimate'] = results.params[electrode]
            
            p_values_cognitive_trust.loc[param_key, 'Intercept_p'] = results.pvalues['Intercept']
            p_values_cognitive_trust.loc[param_key, 'Electrode'] = electrode
            p_values_cognitive_trust.loc[param_key, 'p_value'] = results.pvalues[electrode]

            # Print model summaries
            print(f"Results for {name}, Threshold {threshold}, Electrode {electrode}")
            print(results.summary())

In [ ]:
# Multiverse plot for all electrodes for cognitive_trust

# Function to extract the numeric part from dataset name
def extract_numeric_part(dataset_name):
    match = re.search(r'(\d+(\.\d+)?|\.5)s', dataset_name)
    if match:
        return float(match.group(1))
    return 0

# Extract dataset names and thresholds
try:
    dataset_names = set('_'.join(idx.split('_')[:-2]) for idx in param_estimates_cognitive_trust.index)
    dataset_names = sorted(dataset_names, key=extract_numeric_part)
    thresholds = sorted(set(float(idx.split('_')[-2]) for idx in param_estimates_cognitive_trust.index))
except ValueError as e:
    print(f"Error parsing index: {e}")
    print("Sample problematic indices:", param_estimates_cognitive_trust.index[:5])

# Define font sizes for the plot
tiny_font_size = 6
smaller_font_size = 8  # For super title and column titles

fig, axes = plt.subplots(nrows=len(dataset_names), ncols=2, figsize=(8.5, 11), dpi=300)
fig.subplots_adjust(hspace=0.4, wspace=0.3)
#fig.suptitle("Multiverse: ISC by Electrode", fontsize=smaller_font_size, va='bottom')

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']
colors = ['#bfd3e6', '#8c96c6', '#88419d', '#4d004b']  # colors from https://colorbrewer2.org/

for i, dataset in enumerate(dataset_names):
    epoch_value = f"{extract_numeric_part(dataset)}s"
    for j, df in enumerate([param_estimates_cognitive_trust, p_values_cognitive_trust]):
        ax = axes[i, j]
        for electrode, color in zip(electrodes, colors):
            filtered_data = df[df['Electrode'] == electrode]
            filtered_data = filtered_data[filtered_data.index.str.contains(dataset)]
            filtered_data['Threshold'] = filtered_data.index.map(lambda x: float(x.split('_')[-2]))

            y_column = 'Estimate' if 'Estimate' in df.columns else 'p_value'
            sns.lineplot(x='Threshold', y=y_column, data=filtered_data, ax=ax, marker='o', color=color, label=electrode,
                        linewidth=1.5, markersize=4)

        ax.set_title(f"Epoch {epoch_value}", fontsize=tiny_font_size, pad=3)
        ax.set_xlabel('Filter Threshold', fontsize=tiny_font_size)
        ax.set_ylabel(y_column, fontsize=tiny_font_size)
        ax.tick_params(labelsize=tiny_font_size)
        if j == 1:  # P-value column specific adjustments
            ax.set_ylim(0, 1)
            ax.axhline(0.05, color='red', linestyle='--')
        ax.legend(title='Electrode', title_fontsize='small', fontsize='x-small', loc='upper right')

axes[0, 0].set_title("Parameter Estimate", fontsize=smaller_font_size, pad=20)
axes[0, 1].set_title("P-Value", fontsize=smaller_font_size, pad=20)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('plots/multiverse_isc_by_electrode_cognitive_trust.png', format='png', bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()


In [ ]:
# Storage for multiverse model summaries

param_estimates_affective_trust = pd.DataFrame()
p_values_affective_trust = pd.DataFrame()

In [ ]:
# For Affective Trust

# Apply different filtering thresholds for correlation values (ranging from 0.1 to 1.0 in .1 increments)
# Then, for each filtered threshold, fit mixed models
# Then store parameter estimates and p-values in the new dataframes defined above

# Thresholds
thresholds = [x * 0.1 for x in range(1, 11)]  # 0.1 to 1.0 in increments of 0.1

for name, df in dataframes.items():
    for threshold in thresholds:
        filter_index = ((df[["TP10", "AF7", "AF8", "TP9"]] > threshold).any(1) | 
                        (df[["TP10", "AF7", "AF8", "TP9"]] < -threshold).any(1) | 
                        (df["Signal Drop"] == 1))
        filtered_df = df[~filter_index]

        # Loop through each electrode to fit a model
        for electrode in ['AF7', 'AF8', 'TP9', 'TP10']:
            model = smf.mixedlm(f"affective_trust ~ {electrode}", data=filtered_df, groups=filtered_df["subj"],
                                re_formula="1", vc_formula={"ses": "0 + C(ses)"})
            results = model.fit()

            # Store parameter estimates and p-values
            param_key = f"{name}_{threshold}_{electrode}"
            param_estimates_affective_trust.loc[param_key, 'Intercept'] = results.params['Intercept']
            param_estimates_affective_trust.loc[param_key, 'Electrode'] = electrode
            param_estimates_affective_trust.loc[param_key, 'Estimate'] = results.params[electrode]
            
            p_values_affective_trust.loc[param_key, 'Intercept_p'] = results.pvalues['Intercept']
            p_values_affective_trust.loc[param_key, 'Electrode'] = electrode
            p_values_affective_trust.loc[param_key, 'p_value'] = results.pvalues[electrode]

            # Print model summaries
            print(f"Results for {name}, Threshold {threshold}, Electrode {electrode}")
            print(results.summary())

In [ ]:
# Multiverse plot for all electrodes for affective_trust

# Function to extract the numeric part from dataset name
def extract_numeric_part(dataset_name):
    match = re.search(r'(\d+(\.\d+)?|\.5)s', dataset_name)
    if match:
        return float(match.group(1))
    return 0

# Extract dataset names and thresholds
try:
    dataset_names = set('_'.join(idx.split('_')[:-2]) for idx in param_estimates_affective_trust.index)
    dataset_names = sorted(dataset_names, key=extract_numeric_part)
    thresholds = sorted(set(float(idx.split('_')[-2]) for idx in param_estimates_affective_trust.index))
except ValueError as e:
    print(f"Error parsing index: {e}")
    print("Sample problematic indices:", param_estimates_affective_trust.index[:5])

# Define font sizes for the plot
tiny_font_size = 6
smaller_font_size = 8  # For super title and column titles

fig, axes = plt.subplots(nrows=len(dataset_names), ncols=2, figsize=(8.5, 11), dpi=300)
fig.subplots_adjust(hspace=0.4, wspace=0.3)
#fig.suptitle("Multiverse: ISC by Electrode", fontsize=smaller_font_size, va='bottom')

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']
colors = ['#bfd3e6', '#8c96c6', '#88419d', '#4d004b']  # colors from https://colorbrewer2.org/

for i, dataset in enumerate(dataset_names):
    epoch_value = f"{extract_numeric_part(dataset)}s"
    for j, df in enumerate([param_estimates_affective_trust, p_values_affective_trust]):
        ax = axes[i, j]
        for electrode, color in zip(electrodes, colors):
            filtered_data = df[df['Electrode'] == electrode]
            filtered_data = filtered_data[filtered_data.index.str.contains(dataset)]
            filtered_data['Threshold'] = filtered_data.index.map(lambda x: float(x.split('_')[-2]))

            y_column = 'Estimate' if 'Estimate' in df.columns else 'p_value'
            sns.lineplot(x='Threshold', y=y_column, data=filtered_data, ax=ax, marker='o', color=color, label=electrode,
                        linewidth=1.5, markersize=4)

        ax.set_title(f"Epoch {epoch_value}", fontsize=tiny_font_size, pad=3)
        ax.set_xlabel('Filter Threshold', fontsize=tiny_font_size)
        ax.set_ylabel(y_column, fontsize=tiny_font_size)
        ax.tick_params(labelsize=tiny_font_size)
        if j == 1:  # P-value column specific adjustments
            ax.set_ylim(0, 1)
            ax.axhline(0.05, color='red', linestyle='--')
        ax.legend(title='Electrode', title_fontsize='small', fontsize='x-small', loc='upper right')

axes[0, 0].set_title("Parameter Estimate", fontsize=smaller_font_size, pad=20)
axes[0, 1].set_title("P-Value", fontsize=smaller_font_size, pad=20)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('plots/multiverse_isc_by_electrode_affective_trust.png', format='png', bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()


In [ ]:
# Storage for multiverse model summaries

param_estimates_cognitivetrust_corr = pd.DataFrame()
p_values_cognitivetrust_corr = pd.DataFrame()

In [ ]:
# For cognitivetrust_corr

# Apply different filtering thresholds for correlation values (ranging from 0.1 to 1.0 in .1 increments)
# Then, for each filtered threshold, fit mixed models
# Then store parameter estimates and p-values in the new dataframes defined above

# Thresholds
thresholds = [x * 0.1 for x in range(1, 11)]  # 0.1 to 1.0 in increments of 0.1

for name, df in dataframes.items():
    for threshold in thresholds:
        filter_index = ((df[["TP10", "AF7", "AF8", "TP9"]] > threshold).any(1) | 
                        (df[["TP10", "AF7", "AF8", "TP9"]] < -threshold).any(1) | 
                        (df["Signal Drop"] == 1))
        filtered_df = df[~filter_index]

        # Loop through each electrode to fit a model
        for electrode in ['AF7', 'AF8', 'TP9', 'TP10']:
            model = smf.mixedlm(f"cognitivetrust_corr ~ {electrode}", data=filtered_df, groups=filtered_df["subj"],
                                re_formula="1", vc_formula={"ses": "0 + C(ses)"})
            results = model.fit()

            # Store parameter estimates and p-values
            param_key = f"{name}_{threshold}_{electrode}"
            param_estimates_cognitivetrust_corr.loc[param_key, 'Intercept'] = results.params['Intercept']
            param_estimates_cognitivetrust_corr.loc[param_key, 'Electrode'] = electrode
            param_estimates_cognitivetrust_corr.loc[param_key, 'Estimate'] = results.params[electrode]
            
            p_values_cognitivetrust_corr.loc[param_key, 'Intercept_p'] = results.pvalues['Intercept']
            p_values_cognitivetrust_corr.loc[param_key, 'Electrode'] = electrode
            p_values_cognitivetrust_corr.loc[param_key, 'p_value'] = results.pvalues[electrode]

            # Print model summaries
            print(f"Results for {name}, Threshold {threshold}, Electrode {electrode}")
            print(results.summary())

In [ ]:
# Multiverse plot for all electrodes for cognitivetrust_corr

# Function to extract the numeric part from dataset name
def extract_numeric_part(dataset_name):
    match = re.search(r'(\d+(\.\d+)?|\.5)s', dataset_name)
    if match:
        return float(match.group(1))
    return 0

# Extract dataset names and thresholds
try:
    dataset_names = set('_'.join(idx.split('_')[:-2]) for idx in param_estimates_cognitivetrust_corr.index)
    dataset_names = sorted(dataset_names, key=extract_numeric_part)
    thresholds = sorted(set(float(idx.split('_')[-2]) for idx in param_estimates_cognitivetrust_corr.index))
except ValueError as e:
    print(f"Error parsing index: {e}")
    print("Sample problematic indices:", param_estimates_cognitivetrust_corr.index[:5])

# Define font sizes for the plot
tiny_font_size = 6
smaller_font_size = 8  # For super title and column titles

fig, axes = plt.subplots(nrows=len(dataset_names), ncols=2, figsize=(8.5, 11), dpi=300)
fig.subplots_adjust(hspace=0.4, wspace=0.3)
#fig.suptitle("Multiverse: ISC by Electrode", fontsize=smaller_font_size, va='bottom')

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']
colors = ['#bfd3e6', '#8c96c6', '#88419d', '#4d004b']  # colors from https://colorbrewer2.org/

for i, dataset in enumerate(dataset_names):
    epoch_value = f"{extract_numeric_part(dataset)}s"
    for j, df in enumerate([param_estimates_cognitivetrust_corr, p_values_cognitivetrust_corr]):
        ax = axes[i, j]
        for electrode, color in zip(electrodes, colors):
            filtered_data = df[df['Electrode'] == electrode]
            filtered_data = filtered_data[filtered_data.index.str.contains(dataset)]
            filtered_data['Threshold'] = filtered_data.index.map(lambda x: float(x.split('_')[-2]))

            y_column = 'Estimate' if 'Estimate' in df.columns else 'p_value'
            sns.lineplot(x='Threshold', y=y_column, data=filtered_data, ax=ax, marker='o', color=color, label=electrode,
                        linewidth=1.5, markersize=4)

        ax.set_title(f"Epoch {epoch_value}", fontsize=tiny_font_size, pad=3)
        ax.set_xlabel('Filter Threshold', fontsize=tiny_font_size)
        ax.set_ylabel(y_column, fontsize=tiny_font_size)
        ax.tick_params(labelsize=tiny_font_size)
        if j == 1:  # P-value column specific adjustments
            ax.set_ylim(0, 1)
            ax.axhline(0.05, color='red', linestyle='--')
        ax.legend(title='Electrode', title_fontsize='small', fontsize='x-small', loc='upper right')

axes[0, 0].set_title("Parameter Estimate", fontsize=smaller_font_size, pad=20)
axes[0, 1].set_title("P-Value", fontsize=smaller_font_size, pad=20)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('plots/multiverse_isc_by_electrode_cognitivetrust_corr.png', format='png', bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()


In [ ]:
# Storage for multiverse model summaries

param_estimates_affecttrust_corr = pd.DataFrame()
p_values_affecttrust_corr = pd.DataFrame()

In [ ]:
# For affecttrust_corr

# Apply different filtering thresholds for correlation values (ranging from 0.1 to 1.0 in .1 increments)
# Then, for each filtered threshold, fit mixed models
# Then store parameter estimates and p-values in the new dataframes defined above

# Thresholds
thresholds = [x * 0.1 for x in range(1, 11)]  # 0.1 to 1.0 in increments of 0.1

for name, df in dataframes.items():
    for threshold in thresholds:
        filter_index = ((df[["TP10", "AF7", "AF8", "TP9"]] > threshold).any(1) | 
                        (df[["TP10", "AF7", "AF8", "TP9"]] < -threshold).any(1) | 
                        (df["Signal Drop"] == 1))
        filtered_df = df[~filter_index]

        # Loop through each electrode to fit a model
        for electrode in ['AF7', 'AF8', 'TP9', 'TP10']:
            model = smf.mixedlm(f"affecttrust_corr ~ {electrode}", data=filtered_df, groups=filtered_df["subj"],
                                re_formula="1", vc_formula={"ses": "0 + C(ses)"})
            results = model.fit()

            # Store parameter estimates and p-values
            param_key = f"{name}_{threshold}_{electrode}"
            param_estimates_affecttrust_corr.loc[param_key, 'Intercept'] = results.params['Intercept']
            param_estimates_affecttrust_corr.loc[param_key, 'Electrode'] = electrode
            param_estimates_affecttrust_corr.loc[param_key, 'Estimate'] = results.params[electrode]
            
            p_values_affecttrust_corr.loc[param_key, 'Intercept_p'] = results.pvalues['Intercept']
            p_values_affecttrust_corr.loc[param_key, 'Electrode'] = electrode
            p_values_affecttrust_corr.loc[param_key, 'p_value'] = results.pvalues[electrode]

            # Print model summaries
            print(f"Results for {name}, Threshold {threshold}, Electrode {electrode}")
            print(results.summary())

In [ ]:
# Multiverse plot for all electrodes for affecttrust_corr

# Function to extract the numeric part from dataset name
def extract_numeric_part(dataset_name):
    match = re.search(r'(\d+(\.\d+)?|\.5)s', dataset_name)
    if match:
        return float(match.group(1))
    return 0

# Extract dataset names and thresholds
try:
    dataset_names = set('_'.join(idx.split('_')[:-2]) for idx in param_estimates_affecttrust_corr.index)
    dataset_names = sorted(dataset_names, key=extract_numeric_part)
    thresholds = sorted(set(float(idx.split('_')[-2]) for idx in param_estimates_affecttrust_corr.index))
except ValueError as e:
    print(f"Error parsing index: {e}")
    print("Sample problematic indices:", param_estimates_affecttrust_corr.index[:5])

# Define font sizes for the plot
tiny_font_size = 6
smaller_font_size = 8  # For super title and column titles

fig, axes = plt.subplots(nrows=len(dataset_names), ncols=2, figsize=(8.5, 11), dpi=300)
fig.subplots_adjust(hspace=0.4, wspace=0.3)
#fig.suptitle("Multiverse: ISC by Electrode", fontsize=smaller_font_size, va='bottom')

electrodes = ['AF7', 'AF8', 'TP9', 'TP10']
colors = ['#bfd3e6', '#8c96c6', '#88419d', '#4d004b']  # colors from https://colorbrewer2.org/

for i, dataset in enumerate(dataset_names):
    epoch_value = f"{extract_numeric_part(dataset)}s"
    for j, df in enumerate([param_estimates_affecttrust_corr, p_values_affecttrust_corr]):
        ax = axes[i, j]
        for electrode, color in zip(electrodes, colors):
            filtered_data = df[df['Electrode'] == electrode]
            filtered_data = filtered_data[filtered_data.index.str.contains(dataset)]
            filtered_data['Threshold'] = filtered_data.index.map(lambda x: float(x.split('_')[-2]))

            y_column = 'Estimate' if 'Estimate' in df.columns else 'p_value'
            sns.lineplot(x='Threshold', y=y_column, data=filtered_data, ax=ax, marker='o', color=color, label=electrode,
                        linewidth=1.5, markersize=4)

        ax.set_title(f"Epoch {epoch_value}", fontsize=tiny_font_size, pad=3)
        ax.set_xlabel('Filter Threshold', fontsize=tiny_font_size)
        ax.set_ylabel(y_column, fontsize=tiny_font_size)
        ax.tick_params(labelsize=tiny_font_size)
        if j == 1:  # P-value column specific adjustments
            ax.set_ylim(0, 1)
            ax.axhline(0.05, color='red', linestyle='--')
        ax.legend(title='Electrode', title_fontsize='small', fontsize='x-small', loc='upper right')

axes[0, 0].set_title("Parameter Estimate", fontsize=smaller_font_size, pad=20)
axes[0, 1].set_title("P-Value", fontsize=smaller_font_size, pad=20)

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.savefig('plots/multiverse_isc_by_electrode_affecttrust_corr.png', format='png', bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()


In [ ]:
# Plot the number of participants filtered out at each threshold for each epoch duration

# Initialize storage for filtered counts
filtered_counts = pd.DataFrame()

# Thresholds
thresholds = [x * 0.1 for x in range(1, 11)]  # 0.1 to 1.0 in increments of 0.1

for name, df in dataframes.items():
    for threshold in thresholds:
        filter_index = ((df[["TP10", "AF7", "AF8", "TP9"]] > threshold).any(1) | 
                        (df[["TP10", "AF7", "AF8", "TP9"]] < -threshold).any(1) | 
                        (df["Signal Drop"] == 1))
        filtered_df = df[~filter_index]
        num_filtered = df[filter_index].shape[0]  # Count of filtered observations

        # Store the count of filtered observations
        filtered_counts.loc[f"{name}_{threshold}", 'Filtered Count'] = num_filtered

# Colors array
colors = ['#e0ecf4', '#bfd3e6', '#9ebcda', '#8c96c6', '#8c6bb1', '#88419d', '#810f7c', '#4d004b']


# Plot the filtered counts
fig, ax = plt.subplots(figsize=(8.5, 4.5), dpi=300)
for idx, dataset in enumerate(dataset_names):
    data_subset = filtered_counts[filtered_counts.index.str.contains(dataset)]
    epoch_duration = extract_numeric_part(dataset)  # Extracting the numeric part which is the duration
    # Use modulo to avoid index error in case there are more datasets than colors
    color = colors[idx % len(colors)]
    sns.lineplot(x=[float(idx.split('_')[-1]) for idx in data_subset.index], y='Filtered Count', data=data_subset, ax=ax, 
                 label=f"Epoch {epoch_duration}s", color=color)

ax.set_title('Filtered Observations by Epoch Duration and ISC Threshold Value')
ax.set_xlabel('Threshold')
ax.set_ylabel('Count of Filtered Observations')
ax.legend(title='Dataset')

plt.savefig('plots/filtered_counts_plot.png', format='png', bbox_inches='tight', facecolor='white', edgecolor='none')
plt.show()
